In [498]:
import clingo

from typing import Sequence

In [499]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' ', report_answersets=True):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            if report_answersets:
                print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        mode = "UNKNOWN"
        solve_result = solve_handle.get()
        if solve_result.satisfiable:
            mode = "SAT"
        elif solve_result.unsatisfiable:
            mode = "UNSAT"
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(mode, "{}{}".format(len(models), cardinality_suffix))

    return models

In [500]:
def check(programs: Sequence[str], consistency_checks:str, grounding_context=None):
    print("Run without checks:")
    wo_checks = solve(programs, grounding_context, report_answersets=False)
    print("Run with checks:")
    w_checks = solve((*programs, consistency_checks), grounding_context, report_answersets=False)
    return len(wo_checks) == len(w_checks)

In [501]:
resources = """

resource("Geld").
resource("Essen").
resource("Maschinen Bauteile").
resource("Maschine").

"""
solve([resources], report_answersets=False);

SAT 1


In [502]:
accounts = """
account("Besitzer:Maschinen Bauteile"  ).
account("Besitzer:Maschine" ).
account("Besitzer:Geld").
account("Besitzer:Essen").

account("Arbeiter:Geld").
account("Arbeiter:Essen").

"""
solve([accounts], report_answersets=False);

SAT 1


In [503]:
account_resource = """

account_resource("Besitzer:Maschinen Bauteile", "Maschinen Bauteile").
account_resource("Besitzer:Maschine", "Maschine").
account_resource("Besitzer:Geld", "Geld").
account_resource("Besitzer:Essen", "Essen").

account_resource("Arbeiter:Geld", "Geld").
account_resource("Arbeiter:Essen", "Essen").

"""
solve([account_resource], report_answersets=False);

SAT 1


In [504]:
balance_init = """

obs_at(acc_bal(Acc, 0), ST) :-
  time(ST), not time(ST - 1),
  account(Acc),
  not acc_initBal(Acc, _).
obs_at(acc_bal(Acc, I), ST) :-
  time(ST), not time(ST - 1),
  account(Acc),
  acc_initBal(Acc, I).

"""
solve([balance_init], report_answersets=False);

SAT 1


<block>:4:3-11: info: atom does not occur in any rule head:
  time(ST)

<block>:4:17-29: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:5:3-15: info: atom does not occur in any rule head:
  account(Acc)

<block>:6:7-26: info: atom does not occur in any rule head:
  acc_initBal(#X0,#P1)

<block>:8:3-11: info: atom does not occur in any rule head:
  time(ST)

<block>:8:17-29: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:9:3-15: info: atom does not occur in any rule head:
  account(Acc)

<block>:10:3-22: info: atom does not occur in any rule head:
  acc_initBal(Acc,I)



In [505]:
agents = """

agent("Arbeiter").
agent("Besitzer").

"""
solve([agents], report_answersets=False);

SAT 1


In [506]:
items = """

item("Arbeitszeit").
item(amount_resource(2, "Essen")).
item(amount_resource(2, "Geld")).

"""
solve([items], report_answersets=False);

SAT 1


In [507]:
attainment = """

attainment(action("arbeiten")).
attainment(amount_resource(2, "Essen")).
attainment(amount_resource(2, "Geld")).

"""
solve([attainment], report_answersets=False);

SAT 1


In [508]:
actions = """


action("arbeiten").
action("Maschine verwenden").

action(provide_to_because(Attainment, Agent, Component)) :-
  attainment(Attainment), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_provider(Component, Agent),
  item_attainment(Item, Attainment).

action(allow_from_because(Attainment, Agent, Component)) :-
  attainment(Attainment), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_beneficiary(Component, Agent),
  item_attainment(Item, Attainment).

action(give_fromAcc_to_because(Attainment, Account, Agent, Component)) :-
  attainment(Attainment), account(Account), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_provider(Component, Agent),
  item_attainment(Item, Attainment).

action(accept_intoAcc_from_because(Attainment, Account, Agent, Component)) :-
  attainment(Attainment), account(Account), agent(Agent), component(Component),
  item(Item),
  component_item(Component, Item),
  component_beneficiary(Component, Agent),
  item_attainment(Item, Attainment).

"""
solve([actions], report_answersets=False);

SAT 1


<block>:8:3-25: info: atom does not occur in any rule head:
  attainment(Attainment)

<block>:8:27-39: info: atom does not occur in any rule head:
  agent(Agent)

<block>:8:41-61: info: atom does not occur in any rule head:
  component(Component)

<block>:9:3-13: info: atom does not occur in any rule head:
  item(Item)

<block>:10:3-34: info: atom does not occur in any rule head:
  component_item(Component,Item)

<block>:11:3-39: info: atom does not occur in any rule head:
  component_provider(Component,Agent)

<block>:12:3-36: info: atom does not occur in any rule head:
  item_attainment(Item,Attainment)

<block>:15:3-25: info: atom does not occur in any rule head:
  attainment(Attainment)

<block>:15:27-39: info: atom does not occur in any rule head:
  agent(Agent)

<block>:15:41-61: info: atom does not occur in any rule head:
  component(Component)

<block>:16:3-13: info: atom does not occur in any rule head:
  item(Item)

<block>:17:3-34: info: atom does not occur in any rule head:

In [509]:
action_consequences = """

occ_at(income(1, "Maschine", "Besitzer:Maschine"), T) :-
  obs_at(acc_bal("Besitzer:Maschinen Bauteile", 2), T),
  occ_at(agent_does("Arbeiter", provide_to_because(action("arbeiten"), "Besitzer", Component)), T),
  occ_at(agent_does("Besitzer", allow_from_because(action("arbeiten"), "Arbeiter", Component)), T).

occ_at(expense(2, "Maschinen Bauteile", "Besitzer:Maschinen Bauteile"), T) :-
  obs_at(acc_bal("Besitzer:Maschinen Bauteile", 2), T),
  occ_at(agent_does("Arbeiter", provide_to_because(action("arbeiten"), "Besitzer", Component)), T),
  occ_at(agent_does("Besitzer", allow_from_because(action("arbeiten"), "Arbeiter", Component)), T).

occ_at(expense(1, "Maschine", "Besitzer:Maschine"), T) :-
  obs_at(acc_bal("Besitzer:Maschine", 1), T),
  occ_at(agent_does("Arbeiter", provide_to_because(action("arbeiten"), "Besitzer", Component)), T),
  occ_at(agent_does("Besitzer", allow_from_because(action("arbeiten"), "Arbeiter", Component)), T).

occ_at(income(2, "Maschinen Bauteile", "Besitzer:Maschinen Bauteile"), T) :-
  obs_at(acc_bal("Besitzer:Maschine", 1), T),
  occ_at(agent_does("Arbeiter", provide_to_because(action("arbeiten"), "Besitzer", Component)), T),
  occ_at(agent_does("Besitzer", allow_from_because(action("arbeiten"), "Arbeiter", Component)), T).



"""
solve([action_consequences], report_answersets=False);

SAT 1


<block>:4:3-55: info: atom does not occur in any rule head:
  obs_at(acc_bal("Besitzer:Maschinen Bauteile",2),T)

<block>:5:3-99: info: atom does not occur in any rule head:
  occ_at(agent_does("Arbeiter",provide_to_because(action("arbeiten"),"Besitzer",Component)),T)

<block>:6:3-99: info: atom does not occur in any rule head:
  occ_at(agent_does("Besitzer",allow_from_because(action("arbeiten"),"Arbeiter",Component)),T)

<block>:9:3-55: info: atom does not occur in any rule head:
  obs_at(acc_bal("Besitzer:Maschinen Bauteile",2),T)

<block>:10:3-99: info: atom does not occur in any rule head:
  occ_at(agent_does("Arbeiter",provide_to_because(action("arbeiten"),"Besitzer",Component)),T)

<block>:11:3-99: info: atom does not occur in any rule head:
  occ_at(agent_does("Besitzer",allow_from_because(action("arbeiten"),"Arbeiter",Component)),T)

<block>:14:3-45: info: atom does not occur in any rule head:
  obs_at(acc_bal("Besitzer:Maschine",1),T)

<block>:15:3-99: info: atom does not occu

In [510]:
balance_rules = """

occ_at(transaction(N, Res, From, To, Because), T) :-
  resource(Res), account(From), account(To), time(T),
  occ_at(agent_does(Provider, give_fromAcc_to_because(amount_resource(N, Res), From, Beneficiary, Because)), T),
  occ_at(agent_does(Beneficiary, accept_intoAcc_from_because(amount_resource(N, Res), To, Provider, Because)), T).


occ_at(account_valueChange(Acc, Deposit + Withdrawn + Income + Expense), T) :-
  account(Acc), time(T),
  Income = #sum { N,Res : occ_at(income(N, Res, Acc), T) },
  Expense = #sum { -N,Res : occ_at(expense(N, Res, Acc), T) },
  Withdrawn = #sum { -N,Res,Because : occ_at(transaction(N, Res, Acc, _, Because), T) },
  Deposit = #sum { N,Res,Because : occ_at(transaction(N, Res, _, Acc, Because), T) }.

obs_at(acc_bal(Acc, OldBal + Change), T+1) :-
  account(Acc), time(T+1),
  time(T),
  obs_at(acc_bal(Acc, OldBal), T),
  occ_at(account_valueChange(Acc, Change), T).

"""

In [511]:
contract_domain = """

item_attainment("Arbeitszeit", action("arbeiten")).
item_attainment(amount_resource(2, "Essen"), amount_resource(2, "Essen")).
item_attainment(amount_resource(2, "Geld"), amount_resource(2, "Geld")).

contract("Dienstleistung Tag 1").

component("Dienstleistung Tag 1: Arbeit").
component("Dienstleistung Tag 1: Lohn").

contract_component("Dienstleistung Tag 1", "Dienstleistung Tag 1: Arbeit").
contract_component("Dienstleistung Tag 1", "Dienstleistung Tag 1: Lohn").

component_item("Dienstleistung Tag 1: Arbeit", "Arbeitszeit").
component_item("Dienstleistung Tag 1: Lohn", amount_resource(2, "Geld")).

component_provider("Dienstleistung Tag 1: Arbeit", "Arbeiter").
component_beneficiary("Dienstleistung Tag 1: Arbeit", "Besitzer").

component_provider("Dienstleistung Tag 1: Lohn", "Besitzer").
component_beneficiary("Dienstleistung Tag 1: Lohn", "Arbeiter").

contract("Dienstleistung Tag 2").

component("Dienstleistung Tag 2: Arbeit").
component("Dienstleistung Tag 2: Lohn").

contract_component("Dienstleistung Tag 2", "Dienstleistung Tag 2: Arbeit").
contract_component("Dienstleistung Tag 2", "Dienstleistung Tag 2: Lohn").

component_item("Dienstleistung Tag 2: Arbeit", "Arbeitszeit").
component_item("Dienstleistung Tag 2: Lohn", amount_resource(2, "Geld")).

component_provider("Dienstleistung Tag 2: Arbeit", "Arbeiter").
component_beneficiary("Dienstleistung Tag 2: Arbeit", "Besitzer").
component_provider("Dienstleistung Tag 2: Lohn", "Besitzer").
component_beneficiary("Dienstleistung Tag 2: Lohn", "Arbeiter").

contract("Handel Tag 1").

component("Handel Tag 1: Lieferung").
component("Handel Tag 1: Bezahlung").

contract_component("Handel Tag 1", "Handel Tag 1: Lieferung").
contract_component("Handel Tag 1", "Handel Tag 1: Bezahlung").

component_item("Handel Tag 1: Lieferung", amount_resource(2, "Essen")).
component_item("Handel Tag 1: Bezahlung", amount_resource(2, "Geld")).

component_provider("Handel Tag 1: Lieferung", "Besitzer").
component_beneficiary("Handel Tag 1: Lieferung", "Arbeiter").
component_provider("Handel Tag 1: Bezahlung", "Arbeiter").
component_beneficiary("Handel Tag 1: Bezahlung", "Besitzer").

contract("Handel Tag 2").

component("Handel Tag 2: Lieferung").
component("Handel Tag 2: Bezahlung").

contract_component("Handel Tag 2", "Handel Tag 2: Lieferung").
contract_component("Handel Tag 2", "Handel Tag 2: Bezahlung").

component_item("Handel Tag 2: Lieferung", amount_resource(2, "Essen")).
component_item("Handel Tag 2: Bezahlung", amount_resource(2, "Geld")).

component_provider("Handel Tag 2: Lieferung", "Besitzer").
component_beneficiary("Handel Tag 2: Lieferung", "Arbeiter").
component_provider("Handel Tag 2: Bezahlung", "Arbeiter").
component_beneficiary("Handel Tag 2: Bezahlung", "Besitzer").


"""
solve([contract_domain], report_answersets=False);

SAT 1


In [512]:
contract_static = """

componentState("Initialisiert").
componentState("Gewährleistung").
componentState("Initialisiert [GLE]").
componentState("Gewährleistung [GLE]").

transitionAction("GLE").
transitionAction("LE").

state_transAction_state("Initialisiert","GLE","Initialisiert [GLE]").
state_transAction_state("Initialisiert","LE","Gewährleistung").

state_transAction_state("Initialisiert [GLE]","LE","Gewährleistung [GLE]").

state_transAction_state("Gewährleistung","GLE","Gewährleistung [GLE]").

"""
solve([contract_static], report_answersets=False);

SAT 1


In [513]:
contract_dynamic = """

component_dual(Component, Dual) :-
 component(Component), component(Dual),
 contract(Contract),
 Component != Dual,
 contract_component(Contract, Component), contract_component(Contract, Dual).

% If the transition event does not occur then there is no transition
-transComp_state_at(Component, SuccState, T) :-
  component(Component), componentState(SuccState), time(T),
  componentState(State), transitionAction(TransAction),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, TransAction, SuccState),
  not trans_comp_at(TransAction, Component, T).

% If two states have the same successor state and both states can be reached they will not transition
-transComp_state_at(Component, State1, T) :-
  component(Component), componentState(State1), time(T),
  componentState(State), componentState(State2), transitionAction(TransAction1), transitionAction(TransAction2),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, TransAction1, State1),
  state_transAction_state(State, TransAction2, State2),
  state_transAction_state(State1, TransAction2, SuccState),
  state_transAction_state(State2, TransAction1, SuccState),
  trans_comp_at(TransAction1, Component, T),
  trans_comp_at(TransAction2, Component, T).

% If a transition is not impossible then it will be executed
transComp_state_at(Component, SuccState, T) :-
  component(Component), componentState(SuccState), time(T),
  componentState(State),
  deo_at(component_state(Component, State), T),
  state_transAction_state(State, _, SuccState),
  not -transComp_state_at(Component, SuccState, T).

% INERTIA
deo_at(component_state(Component, State), T+1) :-
  component(Component), componentState(State), time(T+1),
  time(T),
  deo_at(component_state(Component, State), T),
  not transComp_state_at(Component, _, T).

% TRANSITION
deo_at(component_state(Component, SuccState), T+1) :-
  component(Component), componentState(SuccState), time(T+1),
  time(T), componentState(State),
  deo_at(component_state(Component, State), T),
  transComp_state_at(Component, SuccState, T).

%%%
trans_comp_at("LE", Component, T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment), agent(Provider), agent(Beneficiary),
  component_provider(Component, Provider),
  component_beneficiary(Component, Beneficiary),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  occ_at(agent_does(Provider, give_fromAcc_to_because(Attainment, _, Beneficiary, Component)), T),
  occ_at(agent_does(Beneficiary, accept_intoAcc_from_because(Attainment, _, Provider, Component)), T).

trans_comp_at("LE", Component, T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment), agent(Provider), agent(Beneficiary),
  component_provider(Component, Provider),
  component_beneficiary(Component, Beneficiary),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  occ_at(agent_does(Provider, provide_to_because(Attainment, Beneficiary, Component)), T),
  occ_at(agent_does(Beneficiary, allow_from_because(Attainment, Provider, Component)), T).

trans_comp_at("GLE", Component, T) :-
  component(Component), time(T),
  component(Dual),
  component_dual(Component, Dual),
  trans_comp_at("LE", Dual, T).

"""
solve([contract_dynamic], report_answersets=False);

SAT 1


<block>:4:2-22: info: atom does not occur in any rule head:
  component(Component)

<block>:4:24-39: info: atom does not occur in any rule head:
  component(Dual)

<block>:5:2-20: info: atom does not occur in any rule head:
  contract(Contract)

<block>:7:2-41: info: atom does not occur in any rule head:
  contract_component(Contract,Component)

<block>:7:43-77: info: atom does not occur in any rule head:
  contract_component(Contract,Dual)

<block>:11:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:11:25-50: info: atom does not occur in any rule head:
  componentState(SuccState)

<block>:11:52-59: info: atom does not occur in any rule head:
  time(T)

<block>:12:3-24: info: atom does not occur in any rule head:
  componentState(State)

<block>:12:26-55: info: atom does not occur in any rule head:
  transitionAction(TransAction)

<block>:14:3-57: info: atom does not occur in any rule head:
  state_transAction_state(State,TransAction,SuccState)

<block>

In [514]:
contract_init = """

deo_at(component_state(Component, "Initialisiert"), T) :-
  component(Component), time(T),
  contract(Contract),
  contract_component(Contract, Component),
  initContract_at(Contract, T).

"""
solve([contract_init], report_answersets=False);

SAT 1


<block>:4:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:4:25-32: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-21: info: atom does not occur in any rule head:
  contract(Contract)

<block>:6:3-42: info: atom does not occur in any rule head:
  contract_component(Contract,Component)

<block>:7:3-31: info: atom does not occur in any rule head:
  initContract_at(Contract,T)



In [515]:
instance = """

time(1..4).

acc_initBal("Besitzer:Maschinen Bauteile", 2).
initContract_at(Contract, 1) :- contract(Contract).

%% Dienstleistung

occ_at(agent_does("Arbeiter", provide_to_because(action("arbeiten"), "Besitzer", "Dienstleistung Tag 1: Arbeit")), 1).
occ_at(agent_does("Besitzer", allow_from_because(action("arbeiten"), "Arbeiter", "Dienstleistung Tag 1: Arbeit")), 1).

occ_at(agent_does("Arbeiter", provide_to_because(action("arbeiten"), "Besitzer", "Dienstleistung Tag 2: Arbeit")), 2).
occ_at(agent_does("Besitzer", allow_from_because(action("arbeiten"), "Arbeiter", "Dienstleistung Tag 2: Arbeit")), 2).

occ_at(agent_does("Besitzer", give_fromAcc_to_because(amount_resource(2, "Geld"), "Besitzer:Geld", "Arbeiter", "Dienstleistung Tag 1: Lohn")), 3).
occ_at(agent_does("Arbeiter", accept_intoAcc_from_because(amount_resource(2, "Geld"), "Arbeiter:Geld", "Besitzer", "Dienstleistung Tag 1: Lohn")), 3).

occ_at(agent_does("Besitzer", give_fromAcc_to_because(amount_resource(2, "Geld"), "Besitzer:Geld", "Arbeiter", "Dienstleistung Tag 2: Lohn")), 3).
occ_at(agent_does("Arbeiter", accept_intoAcc_from_because(amount_resource(2, "Geld"), "Arbeiter:Geld", "Besitzer", "Dienstleistung Tag 2: Lohn")), 3).

%% Handel

occ_at(agent_does("Besitzer", give_fromAcc_to_because(amount_resource(2, "Essen"), "Besitzer:Essen", "Arbeiter", "Handel Tag 1: Lieferung")), 1).
occ_at(agent_does("Arbeiter", accept_intoAcc_from_because(amount_resource(2, "Essen"), "Arbeiter:Essen", "Besitzer", "Handel Tag 1: Lieferung")), 1).

occ_at(agent_does("Arbeiter", give_fromAcc_to_because(amount_resource(2, "Geld"), "Arbeiter:Geld", "Besitzer", "Handel Tag 1: Bezahlung")), 3).
occ_at(agent_does("Besitzer", accept_intoAcc_from_because(amount_resource(2, "Geld"), "Besitzer:Geld", "Arbeiter", "Handel Tag 1: Bezahlung")), 3).


occ_at(agent_does("Besitzer", give_fromAcc_to_because(amount_resource(2, "Essen"), "Besitzer:Essen", "Arbeiter", "Handel Tag 2: Lieferung")), 2).
occ_at(agent_does("Arbeiter", accept_intoAcc_from_because(amount_resource(2, "Essen"), "Arbeiter:Essen", "Besitzer", "Handel Tag 2: Lieferung")), 2).

occ_at(agent_does("Arbeiter", give_fromAcc_to_because(amount_resource(2, "Geld"), "Arbeiter:Geld", "Besitzer", "Handel Tag 2: Bezahlung")), 3).
occ_at(agent_does("Besitzer", accept_intoAcc_from_because(amount_resource(2, "Geld"), "Besitzer:Geld", "Arbeiter", "Handel Tag 2: Bezahlung")), 3).




"""
solve([instance], report_answersets=False);

SAT 1


<block>:6:33-51: info: atom does not occur in any rule head:
  contract(Contract)



In [516]:
prg = (
    resources,
    accounts,
    agents,
    items,
    attainment,
    actions,
    account_resource,
    action_consequences,
    balance_rules,
    balance_init,
    contract_domain,
    contract_static,
    contract_dynamic,
    contract_init,
    instance,
)

In [517]:
solve((*prg,
       '#show obs_at/2.',
       '#show occ_at/2.',
       '#show deo_at/2.',
       ), sep='\n');

Answer 1: {
deo_at(component_state("Dienstleistung Tag 1: Arbeit","Gewährleistung"),2)
deo_at(component_state("Dienstleistung Tag 1: Arbeit","Gewährleistung"),3)
deo_at(component_state("Dienstleistung Tag 1: Arbeit","Gewährleistung [GLE]"),4)
deo_at(component_state("Dienstleistung Tag 1: Arbeit","Initialisiert"),1)
deo_at(component_state("Dienstleistung Tag 1: Lohn","Gewährleistung [GLE]"),4)
deo_at(component_state("Dienstleistung Tag 1: Lohn","Initialisiert"),1)
deo_at(component_state("Dienstleistung Tag 1: Lohn","Initialisiert [GLE]"),2)
deo_at(component_state("Dienstleistung Tag 1: Lohn","Initialisiert [GLE]"),3)
deo_at(component_state("Dienstleistung Tag 2: Arbeit","Gewährleistung"),3)
deo_at(component_state("Dienstleistung Tag 2: Arbeit","Gewährleistung [GLE]"),4)
deo_at(component_state("Dienstleistung Tag 2: Arbeit","Initialisiert"),1)
deo_at(component_state("Dienstleistung Tag 2: Arbeit","Initialisiert"),2)
deo_at(component_state("Dienstleistung Tag 2: Lohn","Gewährleistung [GLE

In [518]:
consistency_checks = """
:- account_resource(Acc, Res), not resource(Res).
:- account_resource(Acc, Res), not account(Acc).
:- obs_at(acc_bal(Acc, B1), T), obs_at(acc_bal(Acc, B2), T), B1 != B2.
:- acc_initBal(Acc, B), not account(Acc).
:- acc_initBal(Acc, B), B < 0.
"""

In [519]:
check(prg, consistency_checks)

Run without checks:
SAT 1
Run with checks:
SAT 1


True